In [1]:
import numpy as np
import pandas as pd
import csv
from pyitlib import discrete_random_variable as drv
import math
eps = np.finfo(float).eps
from numpy import log2 as log

def Build_Dict(data):
    attributes = {}
    for i in data:
        attr = i.split()[1]
        x = i.split()[2]
        if i.split()[2] == 'NUMERIC':
            field = ["High", "Medium", "Low"]
        else:
            field = x.replace('{','').replace('}','').split(',')
        attributes[attr]=field
    return attributes

dict = {} 
col={}
name={}
labels3={}
prob=np.matrix([[], []])
f = open('Structure.txt','r')
save_file=open('Structure.txt','r')
attributes = Build_Dict(save_file)
a = ['ATTRIBUTE','@']
b= ['NUMERIC']
lst = []
for line in f:
    for word in a:
        if word in line:
            line = line.replace(word,"")
    lst.append(line)
f.close()
f = open("Structure.txt","w")
for line in lst:
    f.write(line)
f = open("Structure.txt", "r")
f = open('Structure.txt','r')
for line in f:
    for word in b:
        if word in line:
            dict.update( {line.split()[0] : "NUMERIC"} )
        if word not in  line:
            col.update( {line.split()[0]: "NOT NUMERIC"} )
name.update(dict) 
name.update(col)
data = pd.read_csv ('train.csv')
test = pd.read_csv ('test.csv')
for key in name:
    if key in dict:
        data.update(data[key].fillna(math.floor(data[key].sum() / data[key].count())))
        test.update(test[key].fillna(math.floor(data[key].sum() / test[key].count())))
    else:
        data.update(data[key].fillna(max(data[key].astype(str))).str.lower())
        test.update(test[key].fillna(max(test[key].astype(str))).str.lower())     
for key in dict:
    data[key] = pd.cut(data[key],3,include_lowest=True,labels=["High", "Medium", "Low"])
    test[key] = pd.cut(test[key],3,include_lowest=True,labels=["High", "Medium", "Low"])
data.to_csv("train.csv",index=False)
test.to_csv("test.csv",index=False)



In [2]:

entropy_node = 0  #Initialize Entropy
values = data["class"].unique()  #Unique objects - 'Yes', 'No'
for value in values:
    fraction = data["class"].value_counts()[value]/len(data["class"])  
    entropy_node += -fraction*np.log2(fraction)
    
def ent(df,attribute):
    target_variables = data["class"].unique()  #This gives all 'Yes' and 'No'
    variables = data[attribute].unique()    #This gives different features in that attribute (like 'Sweet')

    entropy_attribute = 0
    for variable in variables:
        entropy_each_feature = 0
        for target_variable in target_variables:
            num = len(data[attribute][data[attribute]==variable][data["class"] ==target_variable]) #numerator
            den = len(data[attribute][data[attribute]==variable])  #denominator
            fraction = num/(den+eps)  #pi
            entropy_each_feature += -fraction*log(fraction+eps) #This calculates entropy for one feature like 'Sweet'
        fraction2 = den/len(df)
        entropy_attribute += -fraction2*entropy_each_feature   #Sums up all the entropy ETaste

    return(abs(entropy_attribute))

a_entropy = {k:ent(data,k) for k in data.keys()[:-1]}
test.head()


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,previous,poutcome,class
0,Medium,management,married,tertiary,no,High,yes,no,unknown,High,may,High,High,High,unknown,no
1,Medium,technician,single,secondary,no,High,yes,no,unknown,High,may,High,High,High,unknown,no
2,High,entrepreneur,married,secondary,no,High,yes,yes,unknown,High,may,High,High,High,unknown,no
3,Medium,blue-collar,married,unknown,no,High,yes,no,unknown,High,may,High,High,High,unknown,no
4,High,unknown,single,unknown,no,High,no,no,unknown,High,may,High,High,High,unknown,no


In [3]:



def ig(e_dataset,e_attr):
    return(e_dataset-e_attr)


def find_entropy(df):
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy
  
def find_entropy_attribute(df,attribute):
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    target_variables = df[Class].unique()  #This gives all 'Yes' and 'No'
    variables = df[attribute].unique()    #This gives different features in that attribute (like 'Hot','Cold' in Temperature)
    entropy2 = 0
    for variable in variables:
        entropy = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df[Class] ==target_variable])
            den = len(df[attribute][df[attribute]==variable])
            fraction = num/(den+eps)
            entropy += -fraction*log(fraction+eps)
            fraction2 = den/len(df)
            entropy2 += -fraction2*entropy
    return abs(entropy2)

def find_winner(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:
        #Entropy_att.append(find_entropy_attribute(df,key))
        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
    #we return the max value of my entropy verifica tion
    return df.keys()[:-1][np.argmax(IG)]

def bestIGattr(data,attributes,toSplit=False):
    classEntropy = drv.entropy(data['class']).item(0)
    attrsIG = {}
    for attr in attributes:
            attrsIG[attr] = find_entropy(data)-find_entropy_attribute(data,attr)
    maxGain = max(attrsIG.values())
    for attr in attrsIG:
        if attrsIG[attr] == maxGain:
            return attr  




In [4]:
from pprint import pprint

def buildTree(classDict, data, attributes, attrList, toSplit = False,numNodes = 100):
    if(len(data['class'])<=numNodes and len(data['class'])>0):
        return data['class'].mode().iloc[0]
    else:
        if(len(attrList) > 0 ):
            bestOp =bestIGattr(data,attrList,toSplit)
            classDict[bestOp]={}
            for val in attributes[bestOp]:
                if(len(data.loc[data[bestOp] == val]) > 0 and len(attrList) > 0 ):
                    newAttrsList = attrList.copy()
                    newAttrsList.remove(bestOp)
                    classDict[bestOp][val] = buildTree({},data.loc[data[bestOp] == val],attributes,newAttrsList)
            return classDict
        else:
            return data['class'].mode().iloc[0]
    

attrList= list(attributes.keys())
attrList.remove('class')
Decision_tree = buildTree({},data,attributes,attrList)
pprint(Decision_tree)

{'month': {'apr': {'housing': {'no': {'poutcome': {'failure': 'no',
                                                   'other': 'no',
                                                   'success': 'yes',
                                                   'unknown': {'contact': {'cellular': {'marital': {'divorced': 'no',
                                                                                                    'married': {'job': {'admin.': 'no',
                                                                                                                        'blue-collar': 'no',
                                                                                                                        'entrepreneur': 'no',
                                                                                                                        'housemaid': 'no',
                                                                                                                        '

                                                    'student': 'no',
                                                    'technician': {'default': {'no': {'loan': {'no': {'marital': {'divorced': 'no',
                                                                                                                  'married': 'no',
                                                                                                                  'single': 'no'}},
                                                                                               'yes': 'no'}},
                                                                               'yes': 'yes'}},
                                                    'unemployed': 'no',
                                                    'unknown': 'no'}}}},
           'jul': {'duration': {'High': {'poutcome': {'failure': 'no',
                                                      'other': 'no',
                                              

                                                                         'housemaid': {'default': {'no': {'marital': {'divorced': 'no',
                                                                                                                      'married': {'loan': {'no': {'education': {'primary': 'no',
                                                                                                                                                                'secondary': 'no',
                                                                                                                                                                'tertiary': 'no',
                                                                                                                                                                'unknown': 'no'}},
                                                                                                                                           'yes': 'no'}

                                                                                                                              'secondary': {'day': {'High': 'no',
                                                                                                                                                    'Low': 'no',
                                                                                                                                                    'Medium': 'no'}},
                                                                                                                              'tertiary': {'day': {'High': {'loan': {'no': 'no',
                                                                                                                                                                     'yes': 'no'}},
                                                                                                                                                   'Low': 

                                                                       'housemaid': 'no',
                                                                       'management': 'no',
                                                                       'retired': 'no',
                                                                       'self-employed': 'no',
                                                                       'services': 'yes',
                                                                       'student': 'no',
                                                                       'technician': 'yes',
                                                                       'unemployed': 'no',
                                                                       'unknown': 'no'}}}}}},
           'sep': {'poutcome': {'failure': 'no',
                                'other': 'no',
                                'success': 'yes',
                                'unk